# Create Heat Map for significant BCR Signaling genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [40]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [44]:
prot_enr.res2d.iloc[[20]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
20,B cell receptor signaling pathway Homo sapiens...,21/73,0.000301,0.004198,0,0,2.203533,17.867938,MAP2K1;SYK;CHUK;CD81;PRKCB;PIK3CD;NFATC2;PIK3R...,KEGG_2016


# Step 2: Get the list of significant genes 

In [45]:
index = 20
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

B cell receptor signaling pathway Homo sapiens hsa04662
total genes: 21


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [46]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

21

In [47]:
# Only include p-values < a 
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [49]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 600)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [50]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [51]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'SYK', 'CD81', 'PLCG2', 'PIK3R1', 'AKT2'], dtype=object)

In [52]:
# Only include p-values < a 
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [54]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Sig in multiple cancers

In [55]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [57]:
enr2.res2d.iloc[[2]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
2,B cell receptor signaling pathway Homo sapiens...,11/73,1.626924e-08,0.000002,0,0,9.690349,173.786622,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;RAC2;...,KEGG_2016


In [58]:
index2 = 2
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

B cell receptor signaling pathway Homo sapiens hsa04662
total genes: 11


In [59]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['CHUK', 'SYK', 'PRKCB', 'BTK', 'PLCG2', 'PIK3CD', 'INPP5D',
       'PPP3CB', 'BLNK', 'RAC2', 'PIK3CA'], dtype=object)

In [62]:
# Only include p-values < a 
a = 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [64]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=500)

Loading BokehJS ...

In [65]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['SYK', 'PLCG2'], dtype=object)

In [66]:
# Only include p-values < a 
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [69]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=250)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [73]:
#gnm = list(genes_pn_mult.Proteomics.unique())
gnm = genes_mult
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['PPP3CB', 'SYK', 'CHUK', 'PIK3CA', 'PRKCB', 'INPP5D', 'BLNK', 'RAC2', 'BTK', 'PLCG2', 'PIK3CD']


In [74]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,B cell receptor signaling pathway Homo sapiens...,11/73,6.941880e-28,2.033971e-25,0,0,273.972603,17132.824644,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;BTK;R...,KEGG_2016
1,Osteoclast differentiation Homo sapiens hsa04380,8/132,4.710462e-16,6.900827e-14,0,0,110.192837,3888.878834,PPP3CB;SYK;CHUK;PIK3CA;BLNK;BTK;PLCG2;PIK3CD,KEGG_2016
2,Fc epsilon RI signaling pathway Homo sapiens h...,7/68,1.246882e-15,1.217788e-13,0,0,187.165775,6423.179525,SYK;PIK3CA;INPP5D;BTK;RAC2;PLCG2;PIK3CD,KEGG_2016
3,Fc gamma R-mediated phagocytosis Homo sapiens ...,7/93,1.213230e-14,8.886911e-13,0,0,136.852395,4385.148285,SYK;PIK3CA;PRKCB;INPP5D;RAC2;PLCG2;PIK3CD,KEGG_2016
4,Natural killer cell mediated cytotoxicity Homo...,7/135,1.759914e-13,1.031309e-11,0,0,94.276094,2768.732536,PPP3CB;SYK;PIK3CA;PRKCB;RAC2;PLCG2;PIK3CD,KEGG_2016


In [23]:
# Look at p-values 
certain_gene = 'PRPF6'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
212,PRPF6,0.000050,0.391870,Gbm
11890,PRPF6,0.006819,0.140160,Hnscc
27513,PRPF6,0.359535,0.144750,Luad
37369,PRPF6,0.393073,0.107700,Lscc
47188,PRPF6,0.161801,0.068850,Brca
58301,PRPF6,0.338623,-0.006257,Ov
65628,PRPF6,0.000959,-0.193600,En
82250,PRPF6,0.803451,-0.063000,Colon
